<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/08.%20Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 8: Dimensionality Reduction

Bab ini membahas masalah yang dikenal sebagai **kutukan dimensi** (*curse of dimensionality*), di mana banyak hal menjadi aneh dan kontra-intuitif di ruang berdimensi tinggi. Data menjadi sangat tersebar, yang membuat pencarian pola menjadi sulit dan meningkatkan risiko *overfitting*. **Reduksi dimensi** adalah serangkaian teknik untuk mengurangi jumlah fitur dalam dataset, dengan tujuan:
* Mempercepat algoritma training.
* Menghemat ruang penyimpanan.
* Memvisualisasikan data berdimensi tinggi dalam 2D atau 3D.

* **Pendekatan Utama Reduksi Dimensi**:
    * **Proyeksi (*Projection*)**: Memproyeksikan setiap titik data ke *subspace* berdimensi lebih rendah. Bekerja dengan baik jika data terletak di dekat *subspace* yang "datar". **Principal Component Analysis (PCA)** adalah contoh paling populer.
    * **Pembelajaran Manifold (*Manifold Learning*)**: Berasumsi bahwa sebagian besar dataset dunia nyata terletak di dekat *manifold* berdimensi jauh lebih rendah (bentuk yang bisa ditekuk atau dipelintir di ruang dimensi yang lebih tinggi). Tujuannya adalah untuk "membuka" lipatan manifold tersebut. Contohnya adalah **Kernel PCA** dan **Locally Linear Embedding (LLE)**.

* **Principal Component Analysis (PCA)**:
    * PCA mengidentifikasi sumbu-sumbu (*principal components*) yang mempertahankan varians paling besar dalam data.
    * Sumbu pertama adalah arah di mana data paling tersebar, sumbu kedua ortogonal terhadap yang pertama dan mempertahankan sisa varians terbesar, dan seterusnya.
    * **Memilih Jumlah Dimensi**: Daripada memilih jumlah dimensi secara sembarangan, kita bisa memilih jumlah dimensi yang mempertahankan sebagian besar varians (misalnya, 95%).
    * **Varian PCA**:
        * ***Incremental PCA (IPCA)***: Berguna untuk dataset besar yang tidak muat dalam memori (*out-of-core*).
        * ***Randomized PCA***: Algoritma stokastik yang menemukan perkiraan PC dengan cepat.

* **Kernel PCA (kPCA)**: Menerapkan *kernel trick* (yang kita lihat di Bab 5 tentang SVM) ke PCA. Hal ini memungkinkan PCA untuk melakukan proyeksi non-linier, yang sangat baik untuk data yang terletak pada manifold yang kompleks (seperti "Swiss roll").

* **Locally Linear Embedding (LLE)**: Teknik *manifold learning* non-linier lain yang tidak mengandalkan proyeksi. LLE bekerja dengan mengidentifikasi hubungan linier lokal setiap instance dengan tetangga terdekatnya, lalu mencoba mereproduksi hubungan tersebut di ruang berdimensi lebih rendah.

### 1. Principal Component Analysis (PCA)
PCA adalah teknik reduksi dimensi yang paling banyak digunakan. Ia memproyeksikan data ke *hyperplane* yang paling dekat dengan data, sambil memaksimalkan varians.

```python
import numpy as np
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml

# Memuat dataset MNIST (mengambil subset kecil agar cepat)
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

# Membagi data untuk demonstrasi
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# Inisialisasi PCA untuk mempertahankan 95% dari varians
# Scikit-Learn secara otomatis memilih jumlah komponen yang tepat
pca = PCA(n_components=0.95)

# Melakukan reduksi dimensi pada training set
X_train_reduced = pca.fit_transform(X_train)

print("Jumlah komponen setelah PCA (95% varians):", pca.n_components_)
print("Dimensi data asli:", X_train.shape)
print("Dimensi data setelah reduksi:", X_train_reduced.shape)

# Kita bisa merekonstruksi data kembali (dengan sedikit kehilangan informasi)
X_train_recovered = pca.inverse_transform(X_train_reduced)
```
Dengan PCA, kita bisa mengurangi jumlah fitur dari 784 menjadi sekitar 154 (bervariasi) sambil tetap mempertahankan 95% informasi (varians) dari dataset.

### 2. Kernel PCA (kPCA)
kPCA sangat baik untuk dataset non-linier, seperti dataset "Swiss roll".

```python
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt

# Membuat dataset Swiss roll
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

# Menggunakan KernelPCA dengan kernel RBF untuk membuka lipatan manifold
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

# Plot hasil
plt.figure(figsize=(8, 6))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)
plt.title("Hasil kPCA dengan Kernel RBF")
plt.xlabel("$z_1$")
plt.ylabel("$z_2$")
plt.grid(True)
plt.show()
```
Plot di atas menunjukkan bagaimana kPCA berhasil "membuka" gulungan Swiss roll menjadi representasi 2D yang jauh lebih berguna daripada proyeksi linier biasa.

### 3. Locally Linear Embedding (LLE)
LLE adalah pendekatan lain untuk *manifold learning* yang fokus pada hubungan lokal antar instance.

```python
from sklearn.manifold import LocallyLinearEmbedding

# Menggunakan dataset Swiss roll yang sama
# n_neighbors adalah hyperparameter penting yang perlu di-tune
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_reduced_lle = lle.fit_transform(X)

# Plot hasil
plt.figure(figsize=(8, 6))
plt.scatter(X_reduced_lle[:, 0], X_reduced_lle[:, 1], c=t, cmap=plt.cm.hot)
plt.title("Hasil LLE")
plt.xlabel("$z_1$")
plt.ylabel("$z_2$")
plt.grid(True)
plt.show()
```
Sama seperti kPCA, LLE berhasil membuka lipatan Swiss roll. Teknik yang berbeda mungkin bekerja lebih baik pada dataset yang berbeda, jadi seringkali ada baiknya mencoba beberapa pendekatan.
